## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Domino Data Lab Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. You need to make sure that these datasets are uploaded and present in the same directory as this notebook file, since we will be importing these files in Spark and create the following tables under the __`country_club`__ database using Spark SQL.

1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into Spark to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the Domino platform, but for the final deliverable, please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

# Checking Existence of Spark Environment Variables

Make sure your notebook is loaded using a PySpark Workspace. If you open up a regular Jupyter workspace the following variables might not exist

In [1]:
spark

In [2]:
sqlContext

### Run the following if you failed to open a notebook in the PySpark Workspace

This will work assuming you are using Spark in the cloud on domino or you might need to configure with your own spark instance if you are working offline

In [3]:
if 'sc' not in locals():
    from pyspark.context import SparkContext
    from pyspark.sql.context import SQLContext
    from pyspark.sql.session import SparkSession
    
    sc = SparkContext()
    sqlContext = SQLContext(sc)
    spark = SparkSession(sc)

# Create a utility function to run SQL commands

Instead of typing the same python functions repeatedly, we build a small function where you can just pass your query to get results.

- Remember we are using Spark SQL in PySpark
- We can't run multiple SQL statements in one go (no semi-colon ';' separated SQL statements)
- We can run multi-line SQL queries (but still has to be a single statement)

In [4]:
def run_sql(statement):
    try:
        result = sqlContext.sql(statement)
    except Exception as e:
        print(e.desc, '\n', e.stackTrace)
        return
    return result

# Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [5]:
run_sql('drop database if exists country_club cascade')
run_sql('create database country_club')
dbs = run_sql('show databases')
dbs.toPandas()

,databaseName
0,country_club
1,default


# Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you.

To get started, first make sure you have already uploaded the three CSV files and they are present in the same directory as the notebook.

Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [6]:
# File location and type
file_location_bookings = "./Bookings.csv"
file_location_facilities = "./Facilities.csv"
file_location_members = "./Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [7]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
 |-- bookid: integer (nullable = true)
 |-- facid: integer (nullable = true)
 |-- memid: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- slots: integer (nullable = true)

Facilities Schema
root
 |-- facid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- membercost: double (nullable = true)
 |-- guestcost: double (nullable = true)
 |-- initialoutlay: integer (nullable = true)
 |-- monthlymaintenance: integer (nullable = true)

Members Schema
root
 |-- memid: integer (nullable = true)
 |-- surname: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- address: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- telephone: string (nullable = true)
 |-- recommendedby: integer (nullable = true)
 |-- joindate: timestamp (nullable = true)



# Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [8]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [9]:
run_sql('use country_club')
run_sql('REFRESH table bookings')
run_sql('REFRESH table facilities')
run_sql('REFRESH table members')
tbls = run_sql('show tables')
tbls.toPandas()

,database,tableName,isTemporary
0,country_club,bookings,False
1,country_club,facilities,False
2,country_club,members,False


# Test a sample SQL query

__Note:__ You can use multi-line SQL queries (but still a single statement) as follows

In [10]:
result = run_sql('''
                    SELECT * 
                    FROM bookings 
                    LIMIT 3
                 ''')
result.toPandas()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2


# Your Turn: Solve the following questions with Spark SQL

- Make use of the `run_sql(...)` function as seen in the previous example
- You can write multi-line SQL queries but it has to be a single statement (no use of semi-colons ';')
- Make use of the `toPandas()` function as depicted in the previous example to display the query results

#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [11]:
result = run_sql('''
                   SELECT *
                   FROM facilities
                   WHERE membercost > 0
                 ''')
result.toPandas()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,4,Massage Room 1,9.9,80.0,4000,3000
3,5,Massage Room 2,9.9,80.0,4000,3000
4,6,Squash Court,3.5,17.5,5000,80


####  Q2: How many facilities do not charge a fee to members?

In [12]:
result = run_sql('''
                   SELECT count(*)
                   FROM facilities
                   WHERE membercost = 0
                 ''')
result.toPandas()

,count(1)
0,4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [13]:
result = run_sql('''
                   SELECT *
                   FROM facilities
                   WHERE membercost > 0 and membercost < 0.2 * monthlymaintenance
                ''')
result.toPandas()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,4,Massage Room 1,9.9,80.0,4000,3000
3,5,Massage Room 2,9.9,80.0,4000,3000
4,6,Squash Court,3.5,17.5,5000,80


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [14]:
result = run_sql('''
                   SELECT * 
                   FROM facilities
                   WHERE facid in (1,5)
                ''')
result.toPandas()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25.0,8000,200
1,5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [15]:
result = run_sql('''
                   SELECT *, (CASE WHEN monthlymaintenance > 100
                   THEN 'expensive'
                   ELSE 'cheap'
                   END) as label
                   FROM facilities
                 ''')
result.toPandas()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance,label
0,0,Tennis Court 1,5.0,25.0,10000,200,expensive
1,1,Tennis Court 2,5.0,25.0,8000,200,expensive
2,2,Badminton Court,0.0,15.5,4000,50,cheap
3,3,Table Tennis,0.0,5.0,320,10,cheap
4,4,Massage Room 1,9.9,80.0,4000,3000,expensive
5,5,Massage Room 2,9.9,80.0,4000,3000,expensive
6,6,Squash Court,3.5,17.5,5000,80,cheap
7,7,Snooker Table,0.0,5.0,450,15,cheap
8,8,Pool Table,0.0,5.0,400,15,cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [21]:
result = run_sql('''
                    SELECT *
                    FROM members
                    ORDER BY joindate DESC
                 ''')
result.toPandas().iloc[[0]]

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,37,Smith,Darren,"3 Funktown, Denzington, Boston",66796,(822) 577-3541,NaN,2012-09-26 18:08:45


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [33]:
result = run_sql('''
                    SELECT DISTINCT f.name AS facility, 
                    (CASE WHEN b.memid = 0
                    THEN 'Guest'
                    ELSE CONCAT(m.firstname, ' ', m.surname)
                    END) AS member
                    FROM bookings b
                    INNER JOIN facilities f
                    ON b.facid = f.facid
                    INNER JOIN members m
                    ON b.memid = m.memid
                    WHERE f.facid IN (0,1)
                    ORDER BY member, facility
                 ''')
result.toPandas()

,facility,member
0,Tennis Court 1,Anne Baker
1,Tennis Court 2,Anne Baker
2,Tennis Court 1,Burton Tracy
3,Tennis Court 2,Burton Tracy
4,Tennis Court 1,Charles Owen
5,Tennis Court 2,Charles Owen
6,Tennis Court 2,Darren Smith
7,Tennis Court 1,David Farrell
8,Tennis Court 2,David Farrell
9,Tennis Court 1,David Jones


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [34]:
result = run_sql('''
                   SELECT *
                   FROM bookings
                 ''')
result.toPandas()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1
5,5,8,1,2012-07-03 15:00:00,1
6,6,0,2,2012-07-04 09:00:00,3
7,7,0,2,2012-07-04 15:00:00,3
8,8,4,3,2012-07-04 13:30:00,2
9,9,4,0,2012-07-04 15:00:00,2


In [45]:
result = run_sql('''
                   SELECT DISTINCT f.name AS facility, 
                    (CASE WHEN b.memid = 0
                    THEN 'Guest'
                    ELSE CONCAT(m.firstname, ' ', m.surname)
                    END) AS member,
                    (CASE WHEN b.memid != 0
                    THEN f.membercost
                    ELSE f.guestcost
                    END) AS cost
                    FROM bookings b
                    INNER JOIN facilities f
                    ON b.facid = f.facid
                    INNER JOIN members m
                    ON b.memid = m.memid
                    WHERE date_format(b.starttime, 'yyyy-MM-dd') = '2012-09-14'AND
                    (CASE WHEN b.memid != 0
                    THEN f.membercost > 30
                    ELSE f.guestcost > 30
                    END)
                    ORDER BY cost DESC, member, facility
                 ''')
result.toPandas()

,facility,member,cost
0,Massage Room 1,Guest,80.0
1,Massage Room 2,Guest,80.0


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [56]:
result = run_sql('''
                    SELECT DISTINCT f.name AS facility, 
                    (CASE WHEN b.memid = 0
                    THEN 'Guest'
                    ELSE CONCAT(m.firstname, ' ', m.surname)
                    END) AS member,
                    (CASE WHEN b.memid != 0
                    THEN f.membercost
                    ELSE f.guestcost
                    END) AS cost
                    FROM bookings AS b,
                    (SELECT * FROM facilities
                    WHERE membercost > 30 OR guestcost > 30
                    ) AS f,
                    (SELECT * FROM members) AS m
                    WHERE 
                    b.facid = f.facid AND
                    m.memid = b.memid AND
                    date_format(b.starttime, 'yyyy-MM-dd') = '2012-09-14'
                    HAVING cost > 30
                    ORDER BY cost DESC, member, facility
                 ''')
result.toPandas()

,facility,member,cost
0,Massage Room 1,Guest,80.0
1,Massage Room 2,Guest,80.0


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [67]:
result = run_sql('''
                    SELECT f.name AS facility, r.revenue AS revenue
                    FROM facilities f
                    INNER JOIN
                    (SELECT b.facid, SUM(membercost) + SUM(guestcost) AS revenue
                    FROM bookings b
                    INNER JOIN facilities f
                    ON b.facid = f.facid
                    GROUP BY b.facid) AS r
                    USING(facid)
                    WHERE revenue < 1000
                    ORDER BY revenue
                 ''')
result.toPandas()

,facility,revenue
